# Определения токсичности комментариев.

# План Работ:  

- 1.[Подготовка данных.](#1_Подготовка_данных.)
- 2.[Обучение моделей.](#2_Обучение_моделей.)
- 3.[Выводы.](#3_Выводы.)

# Описание данных

- text - текст комментария
- toxic - целевой признак.

# 1. Подготовка данных.<a id='1_Подготовка_данных.'></a>

In [1]:
import pandas as pd

from pymystem3 import Mystem

import re

import numpy as np

import nltk
from nltk.corpus import stopwords as nltk_stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

from sklearn.decomposition import TruncatedSVD

from lightgbm import LGBMClassifier

from catboost import CatBoostClassifier

Загрузим данные:

In [2]:
comments = pd.read_csv('/datasets/toxic_comments.csv')

Посмотрим на общую картину:

In [3]:
comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Пропусков нет.

In [4]:
display(comments.head())
display(comments.tail())

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


,text,toxic
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0
159570,"""\nAnd ... I really don't think you understand...",0


Посмотрим, есть ли дубликаты:

In [5]:
comments.duplicated().sum()

0

Дубликатов нет.  

Начинаем предобрабутку текстов. Напишем функции для лемматизации комментариев и для их очистки от ненужных символов:

In [6]:
ms = Mystem()
def lemmatization_comments(comment):
    lem_comment = ms.lemmatize(comment)
    lem_comment = ''.join(lem_comment)
    return lem_comment.lower()

In [7]:
def clear_comments(comment):
    cl_comment = re.sub(r'[^a-zA-Z]', ' ', comment)
    cl_comment = cl_comment.split()
    return ' '.join(cl_comment)

Добавим в датасет очищенные и лемматизированые тексты:

In [8]:
for i in range(len(comments['text'])):
    comments.loc[i, 'lemm'] = lemmatization_comments(clear_comments(comments.loc[i, 'text']))

In [9]:
display(comments.head(5))
display(comments.tail(5))

,text,toxic,lemm
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he matches this background colour i m se...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not trying to edit war it s...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestions on impr...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...


,text,toxic,lemm
159566,""":::::And for the second time of asking, when ...",0,and for the second time of asking when your vi...
159567,You should be ashamed of yourself \n\nThat is ...,0,you should be ashamed of yourself that is a ho...
159568,"Spitzer \n\nUmm, theres no actual article for ...",0,spitzer umm theres no actual article for prost...
159569,And it looks like it was actually you who put ...,0,and it looks like it was actually you who put ...
159570,"""\nAnd ... I really don't think you understand...",0,and i really don t think you understand i came...


Разобъём данные:

In [10]:
corpus_train, corpus_valid_test, target_train, target_valid_test = train_test_split(comments, comments['toxic'], test_size=0.25, stratify=comments['toxic'], random_state=12345)

In [11]:
corpus_valid, corpus_test, target_valid, target_test = train_test_split(corpus_valid_test, target_valid_test, test_size=0.5, stratify=target_valid_test, random_state=12345)

Подготовим выборку с помощью апсемплинга(и даунсемплинга):

In [12]:
targ_0 = (target_train == 0).sum()
targ_0

107509

In [13]:
targ_1 = (target_train == 1).sum()
targ_1

12169

In [14]:
corpus_train_ups = corpus_train[target_train == 1].sample(targ_0 // 2, replace=True)  # (targ_1 * 3)
corpus_train_ups.shape

(53754, 3)

In [15]:
target_train_ups = pd.Series(np.repeat(1, corpus_train_ups.shape[0]))
target_train_ups.shape

(53754,)

In [16]:
corpus_train_ups = pd.concat([corpus_train_ups, (corpus_train[target_train == 0].sample(corpus_train_ups.shape[0], random_state=12345, replace=True))])
corpus_train_ups.shape

(107508, 3)

In [17]:
target_train_ups = pd.concat([target_train_ups, pd.Series((np.repeat(0, target_train_ups.shape[0])))])
target_train_ups.shape

(107508,)

<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что ты заметил дисбаланс классов и помнишь про методы борьбы с этим явлением.
</div>

Переведём данные в кодировку Unicod:

In [18]:
corpus_train_ups = corpus_train_ups['lemm'].values.astype('U')
corpus_train = corpus_train['lemm'].values.astype('U')
corpus_valid = corpus_valid['lemm'].values.astype('U')
corpus_test = corpus_test['lemm'].values.astype('U')

Подготовим стоп слова:

In [19]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Счётчик со стоп словами:

In [20]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

Посчитаем TF-IDF корпусов train, valid, test:

In [21]:
corpus_train_ups = count_tf_idf.fit_transform(corpus_train_ups)
corpus_train = count_tf_idf.fit_transform(corpus_train)
corpus_valid = count_tf_idf.transform(corpus_valid)
corpus_test = count_tf_idf.transform(corpus_test)

Данные готовы, переходим к обучению.

# 2. Обучение моделей.<a id='2_Обучение_моделей.'></a>

Сначала попробуем Логистическую Регрессию через Грид Сёрч:

In [22]:
param = {'max_iter': [300, 500, 1000], 'C': [3, 5, 7]}
mdel_lr_gs = GridSearchCV(LogisticRegression(solver='sag', class_weight='balanced', random_state=12345), param_grid=param, scoring='f1', cv=5)
mdel_lr_gs.fit(corpus_train, target_train)

Подставляем лучшие параметры и проверяем на валидационной выборке:

In [24]:
model_lr = LogisticRegression(solver='sag', max_iter=300, class_weight={1:2, 0:1}, C=7, random_state=12345)
model_lr.fit(corpus_train, target_train)

LogisticRegression(C=7, class_weight={0: 1, 1: 2}, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=300, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=12345, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)

Проверим модель на валидационной выборке:

In [25]:
pred_valid = model_lr.predict(corpus_valid)
f1_score(target_valid, pred_valid)

0.7863202545068929

f1 = 0.7863202545068929

Результат меры F1 c подобранными параметрами устраивает.  

Попробуем Случайный лес:

In [26]:
mdel_rfc = RandomForestClassifier(n_estimators = 100, max_depth=10, random_state=12345)

In [27]:
mdel_rfc.fit(corpus_train_ups, target_train_ups)

In [28]:
pred_valid = mdel_rfc.predict(corpus_valid)
f1_score(target_valid, pred_valid)

corpus_train_ups = 0.37

F1 Мера на RandomForestClassifier упала слишком низко...  

Возьмём LGBMClassifier:

In [29]:
model_lgbmc = LGBMClassifier(random_state=12345)

In [30]:
model_lgbmc.fit(corpus_train, target_train)  # (corpus_train_ups, target_train_ups)

In [31]:
pred_valid = model_lgbmc.predict(corpus_valid)
f1_score(target_valid, pred_valid)

corpus_train_ups = 0.7315001117818019  
corpus_train = 0.7480474399768585

Результаты гараздо лучше Случайного леса, но немного не дотягивают.

Посмотрим на результаты CatBoost, но перед этим снизим количество признаков:

In [32]:
svd = TruncatedSVD(n_components=500, random_state=12345)
corpus_train_ups2 = svd.fit_transform(corpus_train_ups)
corpus_train2 = svd.fit_transform(corpus_train)
corpus_valid2 = svd.transform(corpus_valid)
corpus_test2 = svd.transform(corpus_test)

In [33]:
model_catb = CatBoostClassifier(random_state=12345, verbose=False)

In [34]:
model_catb.fit(corpus_train2, target_train2)  # (corpus_train_ups, target_train_ups)

In [35]:
pred_valid = model_catb.predict(corpus_valid2)
f1_score(target_valid, pred_valid2)

corpus_train_ups2 = 0.6760503305608873  
corpus_train2 = 0.7034232050893668

Попробуем обучить ComplementNB:

In [36]:
model_cnb = ComplementNB()

In [37]:
model_cnb.fit(corpus_train, target_train)

ComplementNB(alpha=1.0, class_prior=None, fit_prior=True, norm=False)

In [38]:
pred_valid = model_cnb.predict(corpus_valid)
f1_score(target_valid, pred_valid)

0.6055159590951349

f1 = 0.6055159590951349

F1 мера на валидационной выборке после обучения на подготовленной тренировочной выборке.

Наилучшие результаты показала логистическая регрессия, обученная на вектороах, подготовленных с помощью TF-IDF. Протестируем модель на тестовой выборке:

In [39]:
pred = model_lr.predict(corpus_test)
f1_score(target_test, pred)

0.7976969379743521

f1 = 0.7976969379743521

Результаты тестовой выборки немного превосходят условия задания.

# 3. Выводы.<a id='3_Выводы.'></a>

Были предобработаные. Тексты лемматизированы и выброшены стоп слова. Слова превращены в векторы с помощью TF-IDF. Результаты моделей на валидационной выборке:  

- RandomForestClassifier на выборке с апсемплингом(вместе с даунсемплингом) f1 = 0.37.

- ComplementNB = 0.6055159590951349

- CatBoostClassifier = 0.7034232050893668

- LGBMClassifier = 0.7480474399768585

- LogisticRegression = 0.7863202545068929

Исходя из этого, была выбрана логистическая регрессия. Её результаты F1 меры на тестовой выборке = 0.7976969379743521.  

Логистическая регрессия - единственная модель из опробованных, результаты которой удовлетворяют условию задания...

# Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны